In [35]:
# Import libraries
import os
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from contextlib import closing
from utils.utils import SQLiteDB



# Set the display option to show all columns
pd.set_option("display.max_columns", None)

In [36]:
from utils.utils import SQLiteDB

In [37]:
def load_data_from_db(db_path):
    with SQLiteDB(db_path) as db:
        df = db.query("SELECT * FROM BTC_data")
    return df

df = load_data_from_db("../project2/BTC_data.db")



In [38]:
def categorize_and_append_all(df):
    """
    Categorizes multiple columns based on their relation to Bollinger Bands,
    and appends these categories, along with their one-hot encoded versions,
    as new columns to the original DataFrame.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing the required columns.
        
    Returns:
        pd.DataFrame: The original DataFrame with appended binary indicators and one-hot encoded columns.
    """
    # Validate required columns
    band_columns = ["upper_b1", "lower_b1", "upper_b2", "lower_b2", "upper_b3", "lower_b3"]
    indicators_to_categorize = ["close", "vwap", "twap", "parabolicsar", "high", "low"]
    for col in band_columns + indicators_to_categorize:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")
    
    # Iterate through each column to be categorized
    for column in indicators_to_categorize:
        sorted_bands_df = df[band_columns].apply(
            lambda row: sorted(row), axis=1, result_type="expand"
        )
        
        conditions = [
            df[column] > sorted_bands_df.iloc[:, -1],
            (df[column] <= sorted_bands_df.iloc[:, -1]) & (df[column] > sorted_bands_df.iloc[:, -2]),
            (df[column] <= sorted_bands_df.iloc[:, -2]) & (df[column] > sorted_bands_df.iloc[:, -3]),
            (df[column] <= sorted_bands_df.iloc[:, -3]) & (df[column] > sorted_bands_df.iloc[:, 2]),
            (df[column] <= sorted_bands_df.iloc[:, 2]) & (df[column] > sorted_bands_df.iloc[:, 1]),
            (df[column] <= sorted_bands_df.iloc[:, 1]) & (df[column] > sorted_bands_df.iloc[:, 0]),
            df[column] <= sorted_bands_df.iloc[:, 0],
        ]
        
        labels = [
            "above_upper_b3",
            "upper_b3_to_upper_b2",
            "upper_b2_to_upper_b1",
            "upper_b1_to_lower_b1",
            "lower_b1_to_lower_b2",
            "lower_b2_to_lower_b3",
            "below_lower_b3",
        ]
        
        category_column = f"{column}_category"
        df[category_column] = pd.Categorical(
            np.select(conditions, labels, default="uncategorized"),
            categories=labels,
            ordered=True,
        )
        
        # One-hot encode the new category column
        df_one_hot = pd.get_dummies(df[category_column], prefix=category_column)
        df = pd.concat([df, df_one_hot], axis=1)
        
    return df




In [39]:
# Example usage
# Assuming 'df' is your DataFrame containing all the required columns
df = categorize_and_append_all(df)

,index,time,open,high,low,close,vwap,upper_b1,lower_b1,upper_b2,lower_b2,upper_b3,lower_b3,basis,upper,lower,parabolicsar,twap,volume,volume_ma,adx,efi,atr,obv,roc,cci,target_close,hour,day_of_week,USA_open,EU_open,ASIA_open,close_category,close_category_above_upper_b3,close_category_upper_b3_to_upper_b2,close_category_upper_b2_to_upper_b1,close_category_upper_b1_to_lower_b1,close_category_lower_b1_to_lower_b2,close_category_lower_b2_to_lower_b3,close_category_below_lower_b3,vwap_category,vwap_category_above_upper_b3,vwap_category_upper_b3_to_upper_b2,vwap_category_upper_b2_to_upper_b1,vwap_category_upper_b1_to_lower_b1,vwap_category_lower_b1_to_lower_b2,vwap_category_lower_b2_to_lower_b3,vwap_category_below_lower_b3,twap_category,twap_category_above_upper_b3,twap_category_upper_b3_to_upper_b2,twap_category_upper_b2_to_upper_b1,twap_category_upper_b1_to_lower_b1,twap_category_lower_b1_to_lower_b2,twap_category_lower_b2_to_lower_b3,twap_category_below_lower_b3,parabolicsar_category,parabolicsar_category_above_upper_b3,parabolicsar_category_upper_b3_to_upper_b2,parabolicsar_category_upper_b2_to_upper_b1,parabolicsar_category_upper_b1_to_lower_b1,parabolicsar_category_lower_b1_to_lower_b2,parabolicsar_category_lower_b2_to_lower_b3,parabolicsar_category_below_lower_b3,high_category,high_category_above_upper_b3,high_category_upper_b3_to_upper_b2,high_category_upper_b2_to_upper_b1,high_category_upper_b1_to_lower_b1,high_category_lower_b1_to_lower_b2,high_category_lower_b2_to_lower_b3,high_category_below_lower_b3,low_category,low_category_above_upper_b3,low_category_upper_b3_to_upper_b2,low_category_upper_b2_to_upper_b1,low_category_upper_b1_to_lower_b1,low_category_lower_b1_to_lower_b2,low_category_lower_b2_to_lower_b3,low_category_below_lower_b3
0,27,2022-09-01 06:45:00,19990.5,20000.0,19945.0,19975.0,20051.91,20105.41,19998.41,20158.92,19944.90,20212.42,19891.40,20065.58,20180.18,19950.97,20093.64,20068.88,1404.39,1079.08,33.07,-12916.01,68.32,-5436.20,-0.39,-128.94,19933.0,6,3,0,0,1,lower_b1_to_lower_b2,False,False,False,False,True,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,lower_b1_to_lower_b2,False,False,False,False,True,False,False
1,28,2022-09-01 07:00:00,19975.0,19975.0,19876.0,19933.0,20039.11,20102.27,19975.95,20165.43,19912.79,20228.59,19849.63,20054.03,20173.10,19934.95,20086.77,20064.42,3596.46,1220.30,34.29,-32649.60,70.51,-9032.66,-0.56,-174.97,19898.0,7,3,0,0,1,lower_b1_to_lower_b2,False,False,False,False,True,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,lower_b1_to_lower_b2,False,False,False,False,True,False,False,lower_b2_to_lower_b3,False,False,False,False,False,True,False
2,29,2022-09-01 07:15:00,19933.0,19961.5,19878.0,19898.0,20032.13,20099.98,19964.27,20167.84,19896.41,20235.70,19828.56,20040.22,20164.35,19916.10,20074.13,20059.53,2031.82,1298.35,35.41,-38144.50,71.44,-11064.48,-0.89,-170.95,19879.0,7,3,0,0,1,lower_b1_to_lower_b2,False,False,False,False,True,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,lower_b1_to_lower_b2,False,False,False,False,True,False,False,lower_b2_to_lower_b3,False,False,False,False,False,True,False
3,30,2022-09-01 07:30:00,19898.0,19908.0,19771.0,19879.0,20012.46,20097.56,19927.36,20182.67,19842.26,20267.77,19757.16,20028.20,20165.21,19891.19,20062.24,20053.22,4533.63,1486.24,37.20,-45000.84,76.12,-15598.11,-1.15,-210.71,19891.5,7,3,0,0,1,lower_b1_to_lower_b2,False,False,False,False,True,False,False,upper_b1_to_lower_b1,False,False,False,True,False,False,False,upper_b1_to

In [40]:
import pandas as pd

def dropper(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drops specific columns and performs one-hot encoding on certain categorical columns.
    
    Parameters:
        df (pd.DataFrame): The original DataFrame.
        
    Returns:
        pd.DataFrame: The preprocessed DataFrame.
    """
    
    # Columns to drop
    columns_to_drop = [
        "vwap_category",
        "twap_category",
        "parabolicsar_category",
        "high_category",
        "low_category",
        "open",
        "high",
        "low",
        "vwap",
        "twap",
        "parabolicsar",
        "upper_b1",
        "lower_b1",
        "upper_b2",
        "lower_b2",
        "upper_b3",
        "lower_b3",
        "basis",
        "upper",
        "lower",
        "volume",
        "volume_ma",
        "adx",
        "efi",
        "atr",
        "obv",
        "roc",
        "cci",
        "index",
        "close_category"
    ]
    
    # Drop specified columns
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    # Perform one-hot encoding on 'hour' and 'day_of_week'
    df = pd.get_dummies(df, columns=["hour", "day_of_week"], dtype=bool)
    
    return df


In [41]:
# Example usage
# Assuming 'df' is your DataFrame containing all the required columns
df = dropper(df)

In [43]:
def target(df):
    df["price_diff_percentage"] = ((df["target_close"] - df["close"]) / df["close"]) * 100
    conditions = [(df["price_diff_percentage"] > 2), (df["price_diff_percentage"] < -2),
                  (df["price_diff_percentage"] > 0) & (df["price_diff_percentage"] <= 2),
                  (df["price_diff_percentage"] < 0) & (df["price_diff_percentage"] >= -2)]
    choices = [0, 1, 2, 3]
    df["Target"] = pd.Categorical(np.select(conditions, choices, default=np.nan))
    streak = 0
    prev_value = None
    streaks = []
    for value in df["Target"]:
        if value == prev_value:
            streak += 1
        else:
            streak = 1
        streaks.append(streak)
        prev_value = value
    df["Streak"] = pd.Series(streaks, name="Streak")
    df = df.drop(columns=["price_diff_percentage"])
    columns_to_convert = ["USA_open", "EU_open", "ASIA_open"]
    df[columns_to_convert] = df[columns_to_convert].astype(bool)
    df = df.set_index(["time"])
    return df


In [44]:
df = target(df)

,close,target_close,USA_open,EU_open,ASIA_open,close_category_above_upper_b3,close_category_upper_b3_to_upper_b2,close_category_upper_b2_to_upper_b1,close_category_upper_b1_to_lower_b1,close_category_lower_b1_to_lower_b2,close_category_lower_b2_to_lower_b3,close_category_below_lower_b3,vwap_category_above_upper_b3,vwap_category_upper_b3_to_upper_b2,vwap_category_upper_b2_to_upper_b1,vwap_category_upper_b1_to_lower_b1,vwap_category_lower_b1_to_lower_b2,vwap_category_lower_b2_to_lower_b3,vwap_category_below_lower_b3,twap_category_above_upper_b3,twap_category_upper_b3_to_upper_b2,twap_category_upper_b2_to_upper_b1,twap_category_upper_b1_to_lower_b1,twap_category_lower_b1_to_lower_b2,twap_category_lower_b2_to_lower_b3,twap_category_below_lower_b3,parabolicsar_category_above_upper_b3,parabolicsar_category_upper_b3_to_upper_b2,parabolicsar_category_upper_b2_to_upper_b1,parabolicsar_category_upper_b1_to_lower_b1,parabolicsar_category_lower_b1_to_lower_b2,parabolicsar_category_lower_b2_to_lower_b3,parabolicsar_category_below_lower_b3,high_category_above_upper_b3,high_category_upper_b3_to_upper_b2,high_category_upper_b2_to_upper_b1,high_category_upper_b1_to_lower_b1,high_category_lower_b1_to_lower_b2,high_category_lower_b2_to_lower_b3,high_category_below_lower_b3,low_category_above_upper_b3,low_category_upper_b3_to_upper_b2,low_category_upper_b2_to_upper_b1,low_category_upper_b1_to_lower_b1,low_category_lower_b1_to_lower_b2,low_category_lower_b2_to_lower_b3,low_category_below_lower_b3,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,Target,Streak
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-09-01 06:45:00,19975.0,19933.0,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,3.0,1
2022-09-01 07:00:00,19933.0,19898.0,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,3.0,2
2022-09-01 07:15:00,19898.0,19879.0,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,3.0,3
2022-09-01 07:30:00,19879.0,19891.5,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,2.0,1
2022-09-01 07:45:00,19891.5,19924.5,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,True,False,

In [59]:
def main_logic():
    db_file_path = "BTC_data.db"
    table_name = "KNN_data"
    
    if df is not None:
        with SQLiteDB(db_file_path) as db:
            db.create_table(df, table_name)
            query = f"SELECT * FROM {table_name} LIMIT 5;"
            queried_data = db.query(query)
            if queried_data is not None:
                print(queried_data)
    else:
        print("DataFrame could not be read from the CSV file.")

In [60]:
main_logic()

                  time    close  target_close  USA_open  EU_open  ASIA_open  \
0  2022-09-01 06:45:00  19975.0       19933.0         0        0          1   
1  2022-09-01 07:00:00  19933.0       19898.0         0        0          1   
2  2022-09-01 07:15:00  19898.0       19879.0         0        0          1   
3  2022-09-01 07:30:00  19879.0       19891.5         0        0          1   
4  2022-09-01 07:45:00  19891.5       19924.5         0        0          1   

   close_category_above_upper_b3  close_category_upper_b3_to_upper_b2  \
0                              0                                    0   
1                              0                                    0   
2                              0                                    0   
3                              0                                    0   
4                              0                                    0   

   close_category_upper_b2_to_upper_b1  close_category_upper_b1_to_lower_b1  \
0      